In [ ]:
# This script computes SE for each cluster based on the tree
# and test the difference of SE in each clsuter between disease and health group
import sys
sys.path.append('..')
import abd_profile
import os
import pandas as pd
import GCN
import copy
import numpy as np
import tree_util
import se
from scipy.stats import mannwhitneyu
import seaborn as sns 
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import fdrcorrection as fdr
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Load GCN2008, distance and tree newick string
outer = '../result/GCN_fix_tree'
outdir = '../result/GCN_fix_tree/SE/GCN_tree_diff'
ori_GCN = GCN.input_GCN('../data/gcn2008.tsv', transfer=True)
d_df = pd.read_csv('../data/sp_d.tsv', sep='\t', header=0, index_col=0)
with open(os.path.join(outer, 'renamed_GCN_tree.newick')) as fp:
    newick_tree = fp.read()

In [3]:
# recover the tree from newick string
json_tree = tree_util.parse(newick_tree)
largest = {'largest': 0}
leaf_list, l = tree_util.recu_compute(json_tree, 0, largest)
largest_level = largest['largest']
nlayer = largest_level
leaf_list, l = tree_util.recu_compute(json_tree, 0, largest)
layer_leaves_dict = tree_util.make_layer_dict(nlayer)

tree_util.recu_layer(json_tree, layer_leaves_dict)
tree_util.to_layer_leaves(layer_leaves_dict, nlayer)
name_dict = {}
for sp in d_df.columns:
        name_dict[sp] = sp.replace('_', '-')
renamed_GCN = ori_GCN[list(d_df.columns)].rename(columns=name_dict)
renamed_d = d_df.rename(columns=name_dict, index=name_dict)
result = {}
# compute leaf layer
result['leaves_dict'] = copy.deepcopy(layer_leaves_dict)
parent_dict = {}
tree_util.parents(json_tree, parent_dict)
node_leaves = {}
for level in layer_leaves_dict.keys():
    for node, sp_list in layer_leaves_dict[level].items():
        if node in node_leaves.keys():
            continue
        node_leaves[node] = copy.deepcopy(sp_list)
subtree_nodes = {}
for l in leaf_list:
    parent = parent_dict[l]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent].append(l)

for node in node_leaves.keys():
    parent = parent_dict[node]
    if parent not in subtree_nodes.keys():
        subtree_nodes[parent] = []
    subtree_nodes[parent] += subtree_nodes[node]
    subtree_nodes[parent].append(node)

for node in subtree_nodes.keys():
    subtree_nodes[node].append(node)

direct_children_dict = {}
for node, parent in parent_dict.items():
    if parent not in direct_children_dict:
        direct_children_dict[parent] = []
    direct_children_dict[parent].append(node)

result['leaves_dict']['root'] = {'root': list(name_dict.values())}
node_leaves['root'] = list(name_dict.values())

In [4]:
# compute FR network for each sample
def fr(d_df, profile, sname):
    #print(profile.columns, d_df.index)
    sp_list = list(set(profile.columns).intersection(set(d_df.index)))
    #print(sp_list)
    sp_d_df = d_df.loc[sp_list, sp_list]
    #print(profile)
    sp_profile = np.array(profile.loc[sname, sp_list])
    value = np.dot(sp_profile.reshape(len(sp_profile), 1),sp_profile.reshape(1, len(sp_profile)))
    width = value.shape[0]
    cor_df = np.ones(shape=(width, width)) - sp_d_df.values
    for i in range(width):
        cor_df[i][i] = 0
    value = np.multiply(value, cor_df)
    fr_df = pd.DataFrame(value, index=sp_list, columns=sp_list)
    return fr_df

# compute SE for all sample
def multisample_se(profile, d_df, parent_dict, node_leaves, child_dict, direct_children_dict, param):
    result = pd.DataFrame(index=profile.index, columns=(list(node_leaves.keys())))
    for sname in profile.index:
        # print(sname)
        edge_df = fr(d_df, profile, sname)
        tmp = se.subtree_se_adj(edge_df, parent_dict, node_leaves, child_dict, direct_children_dict, param)
        for node in tmp.keys():
                value = tmp[node]
                result.loc[sname, node] = value
    return result


In [5]:
# load profile and metadata
# compute se for each cluster
def main(indir, GCN_df, sp_d, result, name_dict, parent_dict, node_leaves, child_dict, direct_children_dict, param):
    #input dir: including metadata.tsv and abd.tsv
    # output result to dir
    abd_path = os.path.join(indir, 'abd.tsv')
    metadata_path = os.path.join(indir, 'metadata.tsv')

    # first process data from gutmeta
    raw_profile = abd_profile.input_profile(abd_path, transfer=True)
    metadata = pd.read_csv(metadata_path, sep='\t', header=0)
    selected_raw_profile = raw_profile.loc[list(metadata['sample_id']), :]
    crc_profile = abd_profile.check(selected_raw_profile, GCN_df, name_dict)

    pheno_list = {}
    pheno_profiles = {}
    for i in range(metadata.shape[0]):
        pheno = metadata.loc[i, 'disease']
        if pheno not in pheno_list.keys():
            pheno_list[pheno] = []
        if metadata.loc[i, 'sample_id'] in list(crc_profile.index):
            pheno_list[pheno].append(metadata.loc[i, 'sample_id'])

    for c, clist in pheno_list.items():
        pheno_profiles[c] = copy.deepcopy(crc_profile.loc[clist, :])
    # seperate by enterotypes

    
    level_se_result = {}   
    for p, profile in pheno_profiles.items():
        p_result = multisample_se(profile, sp_d, parent_dict, node_leaves, child_dict, direct_children_dict, param)
        for level, cluster_sp_dict in result['leaves_dict'].items():
            if level not in level_se_result.keys():
                level_se_result[level] = {}
            l_se = p_result.loc[:, list(cluster_sp_dict.keys())]
            level_se_result[level][p] = copy.deepcopy(l_se)
 
    return level_se_result

In [6]:
se_result = {}
plist = ['ACVD',
 'BD',
 'CRC',
 'IBD',
 'IGT',
 'CFS',
 'STH',
 'T2D',
 'adenoma',
 'asthma',
 'carcinoma_surgery_history',
 'hypertension',
 'migraine',
 'schizofrenia']
outer = '../data'
param = 0.5


In [7]:
for project in plist:
    projects_dir = os.path.join(outer, project)
    se_result[project] = {}
    for dir in os.listdir(projects_dir):
        indir = os.path.join(projects_dir, dir)
        # print(project, dir)
        level_b = main(indir, renamed_GCN, renamed_d, result, name_dict, parent_dict, node_leaves, subtree_nodes, direct_children_dict, param)
        se_result[project][dir] = copy.deepcopy(level_b)


different sp: {'s__Blastocystis_sp_subtype_1', 's__Saccharomyces_cerevisiae'}
!!!difference species are deleted
different sp: {'s__Blastocystis_sp_subtype_1'}
!!!difference species are deleted
different sp: {'s__Blastocystis_sp_subtype_1'}
!!!difference species are deleted
different sp: {'s__Blastocystis_sp_subtype_1', 's__Saccharomyces_cerevisiae'}
!!!difference species are deleted
different sp: {'s__Saccharomyces_cerevisiae'}
!!!difference species are deleted
different sp: {'s__Blastocystis_sp_subtype_1', 's__Saccharomyces_cerevisiae'}
!!!difference species are deleted
different sp: {'s__Saccharomyces_cerevisiae', 's__Candida_albicans', 's__Candida_tropicalis'}
!!!difference species are deleted
different sp: {'s__Blastocystis_sp_subtype_1', 's__Saccharomyces_cerevisiae'}
!!!difference species are deleted
different sp: {'s__Blastocystis_sp_subtype_1', 's__Saccharomyces_cerevisiae'}
!!!difference species are deleted
different sp: {'s__Blastocystis_sp_subtype_1', 's__Saccharomyces_cerev

In [8]:
# output se to file
def output_se(se_result, outdir, pheno, project):
    odir = os.path.join(outdir, pheno, project)
    if not os.path.exists(odir):
        os.makedirs(odir)
    pheno_df = pd.DataFrame()
    for level, level_result in se_result[pheno][project].items():
        pheno_result = level_result[pheno]
        for node in pheno_result.columns:
            if node not in pheno_df.columns:
                for sname in pheno_result.index:
                    pheno_df.loc[sname, node] = pheno_result.loc[sname, node]
    pheno_df.to_csv(os.path.join(odir, 'se_{}.tsv'.format(pheno)), sep='\t')
    health_df = pd.DataFrame()
    for level, level_result in se_result[pheno][project].items():
        pheno_result = level_result['Health']
        for node in pheno_result.columns:
            if node not in health_df.columns:
                for sname in pheno_result.index:
                    health_df.loc[sname, node] = pheno_result.loc[sname, node]
    health_df.to_csv(os.path.join(odir, 'se_{}.tsv'.format('Health')), sep='\t')

In [9]:
for pheno in se_result.keys():
    for project in se_result[pheno].keys():
        output_se(se_result, outdir, pheno, project)

In [10]:
if not os.path.exists(outdir):
    os.makedirs(outdir)
p_cutoff = 0.05

# test the difference of SE in each cluster between disease and health group
p_df = pd.DataFrame()
mean_df = pd.DataFrame()
valid_dict = {}
show_node = []
for level in result['leaves_dict'].keys():
    for project in plist:
        for dir in se_result[project].keys():
            if project not in se_result[project][dir][level].keys():
                continue
            if 'Health' not in se_result[project][dir][level].keys():
                continue
            if dir not in valid_dict.keys():
                valid_dict[dir] = []
            l_se1 = se_result[project][dir][level][project]
            l_se2 = se_result[project][dir][level]['Health']
            for inode in l_se1.columns:
                if (list(l_se1[inode]).count(0) > len(list(l_se1[inode]))*0.8) and (list(l_se2[inode]).count(0) > len(list(l_se2[inode]))*0.8):
                    continue
                valid_dict[dir].append(inode)
                #print(l_se1[inode])
                # t, p1 = ttest_ind(list(l_se1[inode]), list(l_se2[inode]), equal_var=False)
                t, p1 = mannwhitneyu(list(l_se1[inode]), list(l_se2[inode]))
                p_df.loc[dir, inode] = p1
                mean_df.loc[dir, inode] = np.mean(l_se1[inode]) - np.mean(l_se2[inode])
p_df.fillna(np.nan, inplace=True)

In [11]:
# FDR adjust
for dir in p_df.index:
    valid_nodes = valid_dict[dir]
    p_vector = list(p_df.loc[dir, valid_nodes])
    p_adj = fdr(p_vector, p_cutoff)[1]
    for i, node in enumerate(valid_nodes):
        p_df.loc[dir, node] = p_adj[i]
p_df.to_csv(os.path.join(outdir, 'p_all_cohorts.tsv'), sep='\t')

In [12]:
node_plot = pd.DataFrame()
for dir in p_df.index:
    for inode in p_df.columns:
        p1 = p_df.loc[dir, inode]
        if pd.isna(p1):
            node_plot.loc[dir, inode] = np.nan
        elif p1 < p_cutoff:
            if mean_df.loc[dir, inode] > 0:
                node_plot.loc[dir, inode] = 1
            else:
                node_plot.loc[dir, inode] = -1
        else:
            node_plot.loc[dir, inode] = 0


In [13]:
# plot the result of differential test
show_node = []
for inode in node_plot.columns:
    all_zero = True
    all_nan = True
    for dir in node_plot.index:
        if not pd.isna(node_plot.loc[dir, inode]):
            all_nan = False
        if node_plot.loc[dir, inode] != 0:
            all_zero = False
    if not (all_zero or all_nan):
        show_node.append(inode)
plt.figure(figsize=(10, 6))
part_df = node_plot[show_node]
cluster_name_dict = {}
for inode in part_df.columns:
    cluster_name_dict[inode] = inode.split('_')
    if len(cluster_name_dict[inode])>1:
        cluster_name_dict[inode] = cluster_name_dict[inode][1]
    else:
        cluster_name_dict[inode] = cluster_name_dict[inode][0]
part_df.rename(columns=cluster_name_dict, inplace=True)
sns.heatmap(part_df, vmax=1, vmin=-1, square=True, linecolor='black', cbar=False, xticklabels=True, yticklabels=True, linewidths=1, cmap="coolwarm")

#show_node = sorted(show_node)
sns.heatmap(part_df, vmax=1, vmin=-1, square=True, linecolor='black', cbar=False, xticklabels=True, yticklabels=True, linewidths=1, cmap="coolwarm")
plt.title('Different SE of all cohorts')
plt.xticks(rotation=90)
ax = plt.gca()

ax.spines['top'].set_visible(True)
ax.spines['bottom'].set_visible(True)
ax.spines['left'].set_visible(True)
ax.spines['right'].set_visible(True)
opath = os.path.join(outdir, 'all_cohorts.svg')
plt.tight_layout()
plt.savefig(opath, dpi=300, format='svg')


In [14]:
for disease in plist:   
    show_cohort = []
    show_node = []   
    for cohort in node_plot.index:
        if cohort.startswith(disease):
            show_cohort.append(cohort) 
    part_df = node_plot.loc[show_cohort, :]    
    for inode in part_df.columns:
        all_zero = True
        all_nan = True
        for dir in part_df.index:
            if not pd.isna(node_plot.loc[dir, inode]):
                all_nan = False
            if node_plot.loc[dir, inode] != 0:
                all_zero = False
        if not (all_zero or all_nan):
            show_node.append(inode)
    plt.figure(figsize=(5, 12))
    part_df = part_df[show_node]
    non_zero_count = part_df.astype(bool).sum(axis=0) - part_df.isnull().sum(axis=0)
    sorted_df = part_df.iloc[:, non_zero_count.argsort()]
    #show_node = sorted(show_node)
    sns.heatmap(sorted_df.T, vmax=1, vmin=-1, square=True, linecolor='black', cbar=False, xticklabels=True, yticklabels=True, linewidths=1, cmap="coolwarm")
    plt.title('Different SE of {} cohorts'.format(disease))
    plt.xticks(rotation=90)
    ax = plt.gca()

    ax.spines['top'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['right'].set_visible(True)
    opath = os.path.join(outdir, '{}.pdf'.format(disease))
    plt.tight_layout()
    plt.savefig(opath, dpi=300, format='pdf')
    part_df[show_node].to_csv(os.path.join(outdir, '{}.tsv'.format(disease)), sep='\t')
    #plt.show()
    #print(part_df[show_node].shape)

c:\Users\lijiache2\.conda\envs\meta_fr_win\lib\site-packages\seaborn\matrix.py:309: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))
c:\Users\lijiache2\.conda\envs\meta_fr_win\lib\site-packages\seaborn\matrix.py:309: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))
c:\Users\lijiache2\.conda\envs\meta_fr_win\lib\site-packages\seaborn\matrix.py:309: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))
c:\Users\lijiache2\.conda\envs\meta_fr_win\lib\site-packages\seaborn\matrix.py:309: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformat